In [1]:
import mne
from pymatreader import read_mat

In [2]:
import numpy as np

In [3]:
from pyod.models import hbos, abod

In [4]:
data_folder = mne.datasets.sample.data_path('C:\Учеба\Аспирантура\Кандидатка\matlab_stuff')
data_raw_file = (data_folder / 'data.mat')
data = read_mat(data_raw_file)

C:\Users\coaka\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Для удаления выбросов воспользуемся специальной библиотекой pyod. Pyod содержит в себе несколько десятков моделей,
которые используются для удаления выбросов, начиная от статистических методов и заканчивая сложными моделями DeepLearning.

Для нашего случая удобно воспользоваться моделью из этой библиотеки под названием ABOD. ABOD(Angle-based outlier detection) - модель, базирующася на выделении выбросов по вычислению углу элемента данных, образованному вместе с соседними элементами.
В случае получения экстремального значения угла данные помечаются как выброс.

Для примера рассмотрим триалы первого подопытного. 

In [26]:
trials_variance = list()
first_trial = data['data'][0]['trial']
for frame_trial in first_trial:
    trial_var_per_chanel = frame_trial.var(1)
    trials_variance.append(trial_var_per_chanel)

clf = abod.ABOD(0.05, 10, 'default')
clf.fit(trials_variance)

ABOD(contamination=0.05, method='default', n_neighbors=10)

In [27]:
outliers_indices = list()
for i in range(clf.labels_.shape[0]):
    if clf.labels_[i] == 1:
        outliers_indices.append(i+1)
print(outliers_indices)

[104, 166, 167, 200, 225, 261, 313, 318, 325, 333, 341, 349, 357, 359, 366, 369, 383, 386, 392, 399]


In [28]:
print(len(outliers_indices))

20


In [53]:
trials_variance_second = list()
second_trial = data['data'][1]['trial']
for frame_trial in second_trial:
    trial_var_per_chanel = frame_trial.var(1)
    trials_variance_second.append(trial_var_per_chanel)

In [54]:
second_outliers = clf.predict(trials_variance_second)

In [55]:
outliers_indices = list()
for i in range(second_outliers.shape[0]):
    if second_outliers[i] == 1:
        outliers_indices.append(i+1)
print(outliers_indices)

[55, 58, 97, 103, 107, 126, 137, 162, 176, 182, 184, 203, 204, 210, 236, 237, 256, 286, 296, 298, 317, 318, 319, 320, 336, 337, 338, 358, 359, 366, 380, 381]


Если сравнить данные с теми, что были получены человеком мы увидим довольно высокий процент совпадений